In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [2]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    elif t=='each':
        x=x.dropna().values
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [3]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [4]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [5]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [6]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [30]:
holdings='GDXJ,AMZN,AAPL,FSLY,PTON,FUTU,BLCN,NOMD,GDXJ,BGS,AWK,SHW,EAT,CF,BATT,CAKE,TTEK,NWL,FAST,FXE,URA,ITB,GOLD,SPY,DOCU,ZM,PTON,MSFT,NVDA,EA,CRM,VEEV,ADBE,QQQ,TSLA,W,ESTC,CLX,GDX,CDNS,MELI,ARKF,TTD,WK,SNAP,ESPO,NEM,YETI,FCX'


In [31]:
holdings=holdings.split(',')

In [91]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
holdings=['TSLA']

In [92]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [93]:
holdings

['TSLA']

In [94]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(8).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(5)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


TSLA


In [59]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12]    
             }
            )

In [37]:
res.sort_values(by='k_change',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
17,NWL,2020-08-05,0.521311,-0.350159,-0.473990,14.83,54.63,60.29,0,16.54,1.880534,0.002515,-0.047727
15,CAKE,2020-08-05,0.495021,-0.315858,-0.471821,17.55,53.01,54.70,0,53.54,2.897359,0.023423,-0.091080
34,TSLA,2020-08-05,0.422716,-0.276196,-0.449336,18.39,57.15,61.86,3,0.63,329.886286,-0.028124,-0.009399
44,SNAP,2020-08-05,0.154472,-0.228134,-0.501000,23.51,43.23,32.59,2,2.78,4.140056,-0.040481,-0.052402
39,CDNS,2020-08-05,0.875098,-0.096352,-0.446192,34.32,69.42,91.13,0,97.11,11.717814,0.037241,0.034727
37,CLX,2020-08-05,0.835225,-0.088640,-0.410959,35.03,68.07,91.15,2,91.88,17.031946,0.023746,0.026667
12,EAT,2020-08-05,0.782731,-0.074988,-0.481087,39.19,60.29,76.79,1,95.25,5.065767,0.056527,-0.013728
48,FCX,2020-08-05,0.659888,-0.069981,-0.496007,38.65,66.73,89.08,1,22.66,2.170109,-0.009189,0.050376
7,NOMD,2020-08-05,0.889243,-0.067901,-0.389740,36.99,74.73,96.76,0,96.19,1.114205,0.049178,0.031374
9,BGS,2020-08-05,0.828438,-0.065901,-0.465412,38.50,78.94,98.46,8,98.29,3.482999,0.067250,0.096098


In [39]:
res.sort_values(by='22dayret',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
3,FSLY,2020-08-05,0.931278,0.551064,-0.495274,81.21,68.63,81.63,8,98.70,28.204288,0.178784,0.282317
5,FUTU,2020-08-05,0.940082,0.161144,-0.486721,64.65,82.12,99.13,9,99.72,8.439880,0.160055,0.260951
35,W,2020-08-05,0.966357,-0.017669,-0.458129,44.71,80.29,98.63,8,99.93,59.224381,0.152501,0.246227
42,TTD,2020-08-05,0.962553,0.977631,-0.377694,93.36,71.98,90.32,6,92.21,88.958046,0.041655,0.159289
2,AAPL,2020-08-05,0.907457,0.216588,-0.427017,75.32,77.29,95.24,6,99.90,48.134693,0.071097,0.158065
40,MELI,2020-08-05,0.965807,0.141034,-0.445994,69.17,77.98,97.85,8,99.72,197.099042,0.093476,0.127462
47,YETI,2020-08-05,0.957648,0.028333,-0.456004,55.82,75.63,95.84,6,80.54,9.300500,0.055160,0.102969
9,BGS,2020-08-05,0.828438,-0.065901,-0.465412,38.50,78.94,98.46,8,98.29,3.482999,0.067250,0.096098
6,BLCN,2020-08-05,0.963868,0.003862,-0.436092,48.95,78.34,98.41,3,96.42,3.637889,0.042145,0.085530
41,ARKF,2020-08-05,0.978774,0.045495,-0.322765,60.73,76.83,95.08,6,91.32,5.088254,0.039339,0.082014


In [ ]:
res[(res.RSI_percentile>90) & (res.k_change>-100)].sort_values(by='ret1wk_mean',ascending=False)

In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [ ]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [ ]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [ ]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

In [ ]:
date=list(response.json()['Time Series (Daily)'].keys())

In [ ]:
date.insert(0,response2.json()[0]['date'])

In [ ]:
date

In [ ]:
spy


In [ ]:
np.nanpercentile(spy.k_change,q=10)

In [ ]:
response.json()[0]

In [ ]:
pd.to_datetime('today')

In [ ]:
pd.Timestamp.today().date()

In [ ]:
pd.to_datetime(t[0]).date()

In [ ]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

In [ ]:
spy.adj_close.rolling(90).std().values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

In [95]:
spy=spy.set_index('Date')#['2020-07-01':]

In [96]:
spy['ret']=spy.adj_close/spy.adj_close.shift(5)-1

In [97]:
#spy['ret_3']=spy.ret.rolling(3).mean()

In [98]:
np.nanpercentile(spy.k_change,q=10),np.nanpercentile(spy.ret,q=10)

(-0.44933637984606634, -0.07577710062814896)

In [99]:
def each_percentile(x,val):
    return (x<val).sum()/len(x)

In [100]:
spy['ret_p']=spy.ret.apply(lambda x: each_percentile(spy.ret,x))

In [101]:
#spy['ret_3_p']=spy.ret_3.apply(lambda x: each_percentile(spy.ret,x))

In [102]:
spy['2020-07-01':]

,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret_p
Date,,,,,,,,,,,
2020-07-01,1083.00,1135.3300,1080.5000,1119.63,1119.63,1135.33,858.4400,0.942787,0.254790,0.165250,0.967374
2020-07-02,1221.48,1228.0000,1185.6000,1208.66,1208.66,1228.00,858.4400,0.952411,0.167343,0.225846,0.982704
2020-07-06,1276.69,1377.7900,1266.0400,1371.58,1371.58,1377.79,858.4400,0.959670,0.113715,0.429116,0.995676
2020-07-07,1405.01,1429.5000,1336.7100,1389.86,1389.86,1429.50,866.2000,0.955113,0.013074,0.376985,0.994497
2020-07-08,1405.00,1417.2600,1311.3400,1365.88,1365.88,1429.50,908.5000,0.931853,-0.021584,0.264926,0.987421
2020-07-09,1396.99,1408.5600,1351.2800,1394.28,1394.28,1429.50,908.5000,0.913306,-0.048313,0.245304,0.985849
2020-07-10,1396.00,1548.9200,1376.0100,1544.65,1544.65,1548.92,908.5000,0.934540,-0.021540,0.277986,0.989387
2020-07-13,1659.00,1794.9900,1471.1100,1497.06,1497.06,1794.99,908.5000,0.863218,-0.073655,0.091486,0.889151
2020-07-14,1556.00,1590.0000,1431.0000,1516.80,1516.80,1794.99,908.5000,0.781148,-0.144703,0.091333,0.888758


In [ ]:
spy['2020-02-20':'2020-3-31']


So something can go down at random 
look to see how other stocks have done. if it is aleader like CWH or MELI NEED TO track and buy
that is part of the in-cycle market leader

In [148]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [32]:
fetch_these=['Earnings','Market Cap','P/E','Forward P/E','Sales','Income','Dividend %','Short Float', 'Target Price'
             ,'ROE','Perf Week','Perf Month','Perf Quarter','Perf Year','Insider Own','Insider Trans']

In [33]:

def fetcher(soup, fetch_these,ticker):
    temp=[]
    temp.append(ticker)
    temp.append(pd.datetime.now())
    for i in fetch_these:
        x=soup.find('table', class_='snapshot-table2').find('td', string=i).find_next('td').text
        temp.append(x)
    return temp

In [34]:
stock_stats=[]
for j in holdings:
    try:
        url=str.format('https://finviz.com/quote.ashx?t={}',j)
        r=requests.get(url)
        soup=BeautifulSoup(r.text)
        stock_stats.append(fetcher(soup,fetch_these,j))
        time.sleep(np.random.randint(1,5))
        if np.random.uniform()<=0.3:
            time.sleep(4)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(j, exc_type, exc_tb.tb_lineno)

W <class 'AttributeError'> 7


/Users/rittikghosh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [35]:
stock_stats

[]

In [39]:
j='W'
url=str.format('https://finance.yahoo.com/quote/W?p=W&.tsrc=fin-srch')
r=requests.get(url)

In [41]:
soup=BeautifulSoup(r.text)

In [ ]:
https://cloud.iexapis.com/stable/stock/W/upcoming-earnings?token=pk_9469e57bc642462d905e4dc0003523ab
    http://q4finance.com/analysis/earning%20Drift/BA

In [42]:
url='http://q4finance.com/analysis/earning%20Drift/BA'
r=requests.get(url)

In [44]:
soup=BeautifulSoup(r.text)

In [46]:
soup.find('table', class_='table table-responsive img-fluid  table_text_nowrap').text

AttributeError: 'NoneType' object has no attribute 'text'

In [109]:
url='https://service.q4finance.com/analysis/earning_drift_data'
r=requests.post(url, data={'identifier':'AAPL'})

In [110]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01


In [94]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01


In [154]:
url='https://service.q4finance.com/analysis/primary_info_latest_data'
r=requests.post(url, data={'identifier':'AAPL'})

In [167]:
tickers=pd.read_csv('stock_names.csv')['ticker']

In [168]:
fields=['identifier','ceo','company_url','industry_category','industry_group','sec_filings']

In [169]:
store=[[],[],[],[],[],[]]
c=0
for t in tickers:
    c+=1
    url='https://service.q4finance.com/analysis/primary_info_latest_data'
    r=requests.post(url, data={'identifier':t})
    for idx,f in enumerate(fields):
        store[idx].append(json.loads(r.text)['static_data'][f])
    if c==15:
        time.sleep(2)
        c=0

In [170]:
data=pd.DataFrame({
    'ticker':store[0]
    ,'ceo':store[1]
    ,'website':store[2]
    ,'industry_category':store[3]
    ,'industry_group':store[4]
    ,'sec_filings':store[5]
})

In [171]:
data.to_csv('stock_data.csv')

In [172]:
data

,ticker,ceo,website,industry_category,industry_group,sec_filings
0,A,Michael R. McMullen,www.agilent.com,Health Services,Medical Laboratories & Research,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
1,AA,Roy C. Harvey,www.alcoa.com,Metals & Mining,Aluminum,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
2,AABA,Marissa Ann Mayer,www.yahoo.com,Internet,Internet Information Providers,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
3,AAL,Doug Parker,www.aa.com,Transportation,Major Airlines,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
4,AAN,John W. Robinson,www.aarons.com,Diversified Services,Rental & Leasing Services,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
...,...,...,...,...,...,...
2377,EB,N/A,N/A,nm,nm,https://www.sec.gov/cgi-bin/browse-edgar?CIK=E...
2378,AMSC,Daniel Patrick McGahn,www.amsc.com,Manufacturing,Industrial Electrical Equipment,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
2379,RRD,Daniel L. Knotts,www.rrdonnelley.com,Diversified Services,Business Services,https://www.sec.gov/cgi-bin/browse-edgar?CIK=R...
2380,LE,Jerome Squire Griffith,www.landsend.com,Retail,Catalog & Mail Order Houses,https://www.sec.gov/cgi-bin/browse-edgar?CIK=L...


In [140]:
for f in fields:
    print(json.loads(r.text)['static_data'][f])

AAPL
Tim Cook
www.apple.com
Consumer Durables
Electronic Equipment
https://www.sec.gov/cgi-bin/browse-edgar?CIK=AAPL&owner=exclude&action=getcompany


In [130]:
store=[[],[]]
for etf in json.loads(r.text)['data']:
    store[0].append(etf['identifier'])
    store[1].append(etf['name'])

In [132]:
pd.DataFrame(zip(store[0],store[1]),columns=['ticker','name']).to_csv('stock_names.csv')